<summary>
    <font size="5" color="gray"><b> CONVENIO CENACE-UNISON 2023-2025
 </b></font>
</summary><br>
<summary>
    <font size="4" color="orange"><b> DESARROLLO DE PROTOTIPO: METODOLOGÍA CON APRENDIZAJE PROFUNDO PARA EL PRONÓSTICO A CORTO PLAZO DE DEMANDA DE ENERGÍA CON DATOS EN ADELANTO.
 </b></font>
</summary>

---

<summary>
    <font size="8" color="gray"><b> Libreta de cálculo de error.
 </b></font>
</summary>

---
    
OLIVIA CAROLINA GUTÚ OCAMPO

REYNA YANET HERNÁNDEZ MADA

JULIO WAISSMAN VILANOVA (Responsable del proyecto)


---


<summary>
    <font size="4" color="orange"><b>Libreta de cálculo de error



 </b></font>
</summary>

<summary>
    <font size="2" color="gray"><b> Objetivo

 </b></font>
</summary>

Se toman los archivos originados de cada una de las versiones de los escenarios de prueba del pronosticador tanto en su versión de verano como de invierno.<br>
(SV) Sin variables en adelanto.<br>
(PC) Solo la primera componente en adelanto.<br>
(CED) Solo las variables de calendario en adelanto.<br>
(PCAL) Primera componente y variables de festivos en adelanto.<br>
(ALL) Primera componente, festivos y dem´as de variables de
calendario i.e. día, hora y mes, en adelanto.<br>




<summary>
    <font size="4" color="orange"><b>Proceso



 </b></font>
</summary>

<summary>
    <font size="2" color="gray"><b> Input

 </b></font>
</summary>

Se cargan los 5 archivos generados en cada libreta después del entrenamiento del modelo, se integran a un dataframe para poder verificar sus características y gráficarlos.<br>
Se incluye el escenario de verano, se replica para escenario de Invierno.


In [1]:
import pandas as pd
import numpy as np
import plotly.express as px


In [2]:
import os
#os.chdir('..')
vai = os.getcwd()
print(vai)

/Users/eeevye/ymn/cenace_local/parte_dos/energia_parte2/compare


In [3]:
df = pd.read_csv("./summer_acarusmay24-1/mape_modelo01sinvariables_verano100.csv")
df2 = pd.read_csv("./summer_acarusmay24-1/mape_modelo2_PCEA_summer.csv")
df3 = pd.read_csv("./summer_acarusmay24-1/mape_modelo3_CALADsummer.csv")
df4 = pd.read_csv("./summer_acarusmay24-1/mape_modelo4_PC_FEST_summer.csv")
df5 = pd.read_csv("./summer_acarusmay24-1/mape_modelo5_TEDsummer.csv")
df3.head(2)

,Unnamed: 0,MAPE (%)
0,0,6.764924
1,1,11.993807


In [4]:
df.rename(columns ={"MAPE (%)":"SV"} , inplace = True)
df2.rename(columns ={"MAPE (%)":"PC"} , inplace = True)
df3.rename(columns ={"MAPE (%)":"CED"} , inplace = True)
df4.rename(columns ={"MAPE (%)":"PCAL"} , inplace = True)
df5.rename(columns ={"MAPE (%)":"ALL"} , inplace = True)


In [5]:
#remover columna unnamed
frames = [df, df2, df3, df4, df5]

for df in frames:
    if 'Unnamed: 0' in df.columns:
        del df['Unnamed: 0']

In [6]:
complete = pd.concat(frames, axis =1 ,  join = 'inner') #concatenar versiones del modelo


In [7]:
complete.describe()

,SV,PC,CED,PCAL,ALL
count,529.000000,529.000000,529.000000,529.000000,529.000000
mean,4.101555,2.881709,5.495066,2.632776,2.775860
std,2.026461,1.422536,2.632175,1.161915,0.929442
min,0.957868,0.670625,1.001541,0.772685,0.929578
25%,2.603531,1.773734,3.264471,1.664553,2.144091
50%,3.627659,2.568033,5.026141,2.571235,2.637115
75%,5.028166,3.639906,7.355855,3.353038,3.405867
max,11.525841,8.801503,12.624754,6.068772,6.009987


In [8]:
cols = complete.columns

In [9]:
complete_long = pd.melt(complete, var_name='Model', value_name='MAPE')


In [10]:
#generar gráfico de violin
color_palette = px.colors.qualitative.Vivid

fig = px.violin(complete_long, y='MAPE', x='Model', color='Model', box=True,
                color_discrete_sequence=color_palette)
fig.update_layout(title_text="Distribución MAPE para las 5 variables del modelo<br><i> Verano")


fig.show()

#fig.write_image("MAPE_SUMMER_Violin.pdf")



In [11]:
import os
if not os.path.exists("images"):
    os.mkdir("images")

In [12]:
fig.write_image("images/fig1_summer.pdf")

In [13]:
#graficos de caja
color_palette = px.colors.qualitative.Vivid

fig = px.box(complete_long, y='MAPE', x='Model', color='Model',
             color_discrete_sequence=color_palette)

fig.update_layout(title_text="Distribución MAPE para las 5 variables del modelo<br><i> Verano")

fig.show()

fig.write_image("images/MAPE_SUMMER_BOX.pdf")

### intervalos de MAPE para verano

In [14]:
#generar bins para comparativos de resultados por variante del modelo
bins = [-np.inf, 1, 2, 3, 4, 5, np.inf]
labels = ['0 to 1', '1 to 2', '2 to 3','3 to 4', '4 to 5', 'Beyond 5']

counts_ = {col: pd.cut(complete[col], bins=bins, 
                                 labels=labels, 
                                 right=False).value_counts().sort_index() for col in complete}

counts_df = pd.DataFrame(counts_)
counts_df

,SV,PC,CED,PCAL,ALL
0 to 1,2,8,0,7,2
1 to 2,44,169,9,201,109
2 to 3,129,143,93,133,224
3 to 4,127,123,88,125,143
4 to 5,92,38,71,35,40
Beyond 5,135,48,268,28,11


In [15]:
sum_row = counts_df.sum()

# Append the sum row to the DataFrame
counts_df.loc['Sum'] = sum_row

In [16]:
counts_df

,SV,PC,CED,PCAL,ALL
0 to 1,2,8,0,7,2
1 to 2,44,169,9,201,109
2 to 3,129,143,93,133,224
3 to 4,127,123,88,125,143
4 to 5,92,38,71,35,40
Beyond 5,135,48,268,28,11
Sum,529,529,529,529,529


In [17]:
complete.describe().round(2)


,SV,PC,CED,PCAL,ALL
count,529.00,529.00,529.00,529.00,529.00
mean,4.10,2.88,5.50,2.63,2.78
std,2.03,1.42,2.63,1.16,0.93
min,0.96,0.67,1.00,0.77,0.93
25%,2.60,1.77,3.26,1.66,2.14
50%,3.63,2.57,5.03,2.57,2.64
75%,5.03,3.64,7.36,3.35,3.41
max,11.53,8.80,12.62,6.07,6.01


In [18]:
complete.columns

Index(['SV', 'PC', 'CED', 'PCAL', 'ALL'], dtype='object')

In [19]:
counts_and_percents = {
    col: pd.cut(complete[col], bins=bins, labels=labels, right=False)
    .value_counts(normalize=True)  # This gives the percentage
    .sort_index()
    .multiply(100)   
    .round(2)  # round to 2
    .astype(str) + '%'  # Convert to string and append '%'
    for col in complete
}
# Dataframe para conteo y %
counts_df = pd.DataFrame(counts_and_percents)
counts_df


,SV,PC,CED,PCAL,ALL
0 to 1,0.38%,1.51%,0.0%,1.32%,0.38%
1 to 2,8.32%,31.95%,1.7%,38.0%,20.6%
2 to 3,24.39%,27.03%,17.58%,25.14%,42.34%
3 to 4,24.01%,23.25%,16.64%,23.63%,27.03%
4 to 5,17.39%,7.18%,13.42%,6.62%,7.56%
Beyond 5,25.52%,9.07%,50.66%,5.29%,2.08%


In [20]:
def calculate_outliers_and_percentage(column):
    Q1 = column.quantile(0.25)
    Q3 = column.quantile(0.75)
    IQR = Q3 - Q1
    upper_fence = Q3 + 1.5 * IQR
    outliers = column[column > upper_fence]
    percentage_of_outliers = len(outliers) / len(column) * 100
    return upper_fence, len(outliers), percentage_of_outliers

# Loop through each column col1 to col5 and perform calculations
for col in ['SV', 'PC', 'CED', 'PCAL', 'ALL']:
    upper_fence, number_of_outliers, percentage_of_outliers= calculate_outliers_and_percentage(complete[col])
    print(f"{col} - Límite superior: {upper_fence:.2f}, % de valores atipicos: {percentage_of_outliers:.2f}%")
    

SV - Upper Fence: 8.665117855250218, % of Outliers: 3.21%
PC - Upper Fence: 6.439163553028983, % of Outliers: 2.84%
CED - Upper Fence: 13.492930701134238, % of Outliers: 0.00%
PCAL - Upper Fence: 5.885766082521408, % of Outliers: 0.57%
ALL - Upper Fence: 5.298530876414295, % of Outliers: 2.08%


### Invierno

In [21]:
df_w = pd.read_csv("./wintermay/mape_19MAY01sinvariables_wintr100.csv")
df2_w = pd.read_csv("./wintermay/mape_19MAY2_PCEA_wintr.csv")
df3_w = pd.read_csv("./wintermay/mape_19MAY3_CALADwintr.csv")
#compare/mape_modelo3_CALAD.csv
df4_w = pd.read_csv("./wintermay/mape_19MAY4_PC_FEST_wintr.csv")
df5_w = pd.read_csv("./wintermay/mape_19MAY5_TEDwintr.csv")


In [22]:
df_w.rename(columns ={"MAPE (%)":"SV"} , inplace = True)
df2_w.rename(columns ={"MAPE (%)":"PC"} , inplace = True)
df3_w.rename(columns ={"MAPE (%)":"CED"} , inplace = True)
df4_w.rename(columns ={"MAPE (%)":"PCAL"} , inplace = True)
df5_w.rename(columns ={"MAPE (%)":"ALL"} , inplace = True)


In [23]:
frames_w = [df_w, df2_w, df3_w, df4_w, df5_w]

for df in frames_w:
    if 'Unnamed: 0' in df.columns:
        del df['Unnamed: 0']

In [24]:
complete_w = pd.concat(frames_w, axis =1 ,  join = 'inner')

In [25]:
complete_w

,SV,PC,CED,PCAL,ALL
0,1.289397,1.802388,1.441402,1.526953,1.558985
1,1.653196,2.064661,2.050173,1.552152,1.643911
2,1.817875,2.127696,2.092589,1.531349,1.636937
3,1.934656,2.223989,2.234884,1.571763,1.725045
4,2.123272,2.395179,2.390515,1.668587,1.819561
...,...,...,...,...,...
524,1.210389,1.550173,1.556424,1.479729,1.227461
525,1.239632,1.556246,1.448782,1.372545,1.351336
526,1.372452,1.666115,1.383915,1.390751,1.347173
527,1.295300,1.785434,1.403489,1.444505,1.336974


In [26]:
cols_w = complete_w.columns
#complete_long_w = pd.melt(complete_w, var_name='Model', value_name='MAPE')

In [27]:
complete_long_w = pd.melt(complete_w, var_name='Model', value_name='MAPE')

In [28]:
color_palette = px.colors.qualitative.Vivid

fig_2 = px.violin(complete_long_w, y='MAPE', x='Model', color='Model', box=True,
                color_discrete_sequence=color_palette)
fig_2.update_layout(title_text="Distribución MAPE para las 5 variables del modelo<br><i> Invierno")

fig_2.show()
fig_2.write_image("MAPE_WINTER.pdf")



In [38]:

color_palette = px.colors.qualitative.Vivid

fig = px.box(complete_long_w, y='MAPE', x='Model', color='Model',
             color_discrete_sequence=color_palette)

fig.update_layout(title_text="Distribución MAPE para las 5 variables del modelo<br><i> Invierno")

fig.show()

fig.write_image("images/MAPE_WINTERN_BOX.pdf")

### crear intervalos de MAPE para invierno

In [31]:
bins = [-np.inf, 1, 2, 3, 4, 5, np.inf]
labels = ['0 to 1', '1 to 2', '2 to 3','3 to 4', '4 to 5', 'Beyond 5']


counts_w = {col: pd.cut(complete_w[col], bins=bins, 
                                 labels=labels, 
                                 right=False).value_counts().sort_index() for col in complete_w}

counts_dfw = pd.DataFrame(counts_w)


counts_dfw


,SV,PC,CED,PCAL,ALL
0 to 1,136,74,108,142,145
1 to 2,307,342,336,299,286
2 to 3,78,85,83,63,56
3 to 4,8,28,2,19,41
4 to 5,0,0,0,6,1
Beyond 5,0,0,0,0,0


In [32]:
counts_and_percents_w = {
    col: pd.cut(complete_w[col], bins=bins, labels=labels, right=False)
    .value_counts(normalize=True)  # This gives the percentage
    .sort_index()
    .multiply(100)   
    .round(2) 
    .astype(str) + '%'  # Convert to string and append '%'
    for col in complete_w
}

#Dataframe para conteos y %
counts_dfw = pd.DataFrame(counts_and_percents_w)

counts_dfw

,SV,PC,CED,PCAL,ALL
0 to 1,25.71%,13.99%,20.42%,26.84%,27.41%
1 to 2,58.03%,64.65%,63.52%,56.52%,54.06%
2 to 3,14.74%,16.07%,15.69%,11.91%,10.59%
3 to 4,1.51%,5.29%,0.38%,3.59%,7.75%
4 to 5,0.0%,0.0%,0.0%,1.13%,0.19%
Beyond 5,0.0%,0.0%,0.0%,0.0%,0.0%


In [33]:
complete_w.describe()


,SV,PC,CED,PCAL,ALL
count,529.000000,529.000000,529.000000,529.000000,529.000000
mean,1.391660,1.568439,1.388948,1.432297,1.514844
std,0.589042,0.661742,0.530761,0.737322,0.738361
min,0.568582,0.664114,0.531954,0.513375,0.677342
25%,0.995818,1.128039,1.044904,0.974411,0.959948
50%,1.209439,1.373128,1.270078,1.240104,1.308618
75%,1.636880,1.850763,1.572806,1.609097,1.775285
max,3.383602,3.693319,3.039052,4.282539,4.100638


In [37]:

for col in ['SV', 'PC', 'CED', 'PCAL', 'ALL']:
    upper_fence, number_of_outliers, percentage_of_outliers= calculate_outliers_and_percentage(complete_w[col])
    print(f"{col} - Límite superior: {upper_fence:.2f}, % de valores atípicos: {percentage_of_outliers:.2f}%")

SV - Upper Fence: 2.60, % of Outliers: 6.62%
PC - Upper Fence: 2.93, % of Outliers: 6.24%
CED - Upper Fence: 2.36, % of Outliers: 8.88%
PCAL - Upper Fence: 2.56, % of Outliers: 9.07%
ALL - Upper Fence: 3.00, % of Outliers: 7.94%
